<a href="https://colab.research.google.com/github/JoaoADuarte/ChatBot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

!apt-get update
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver
!pip install selenium
!pip install transformers datasets peft

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,770 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,237 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Package

In [34]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Configurar o ChromeDriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

# Simular um navegador real com cabeçalhos personalizados
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
chrome_options.add_argument(f"user-agent={user_agent}")
driver = webdriver.Chrome(options=chrome_options)

conn = sqlite3.connect("chatbot_jardinagem_db")
cursor = conn.cursor()

# Nossa tabela
cursor.execute("""
CREATE TABLE IF NOT EXISTS artigos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    fonte TEXT,
    titulo TEXT,
    conteudo TEXT,
    tags TEXT,
    categoria TEXT,  -- Aqui é a coluna categoria
    data_extração TEXT
);
""")
conn.commit()
print("DB criado!")


# Insere dados no BD
def inserir_dados(fonte, titulo, conteudo, tags):
  cursor.execute("""
  INSERT INTO artigos (fonte, titulo, conteudo, tags)
  VALUES (?,?,?,?)
  """, (fonte, titulo, conteudo, tags))
  conn.commit()

# Busca os dados do nosso DB
def buscar_conteudo(palavra_chave):
    cursor.execute("""
    SELECT titulo, conteudo FROM artigos
    WHERE tags LIKE ? OR conteudo LIKE ?
    """, (f"%{palavra_chave}%", f"%{palavra_chave}%"))
    resultados = cursor.fetchall()
    return resultados



# URLs das páginas para scraping
urls = [
    "https://eos.com/pt/blog/como-plantar-tomates/",
    "https://www.jardineiro.net/como-plantar-tomate-os-segredos-do-plantio-ate-a-colheita.html",
    "https://www.petz.com.br/blog/plantas/como-plantar-tomate/"
]

for url in urls:
    driver.get(url)
    time.sleep(3)  # Esperar a página carregar


    # Forçando atualização da página para evitar que reutilize dados
    driver.refresh()
    time.sleep(3)
    print(f"\n Extraindo dados de: {url}")

    # Título do artigo, onde pega o "h1" do HTML como título
    try:
      titulo_element = driver.find_element(By.TAG_NAME, "h1")
      titulo = titulo_element.text
    except:
      titulo = "Título não encontrado"

# Extrair texto principal do site (ajustar conforme necessário)
    elementos_texto_p = driver.find_elements(By.TAG_NAME, "p")  # Pegar todos os parágrafos
    conteudo = "\n".join([e.text for e in elementos_texto_p if e.text.strip()]) # elementos_texto alterado para elementos_texto_pif e.text.strip()])

    # Mostrar trecho do conteúdo extraído
    print(f"{titulo} - {conteudo[:300]}...\n")

    # Gerar tags com base no título de forma automática
    tags = ", ".join(titulo.lower().split())

    # Inserir dados no BD
    inserir_dados(url, titulo, conteudo, tags)
    print(f"Dados inseridos com sucesso em {url}")

# Fechar o navegador
driver.quit()

# Testando a busca
palavra_chave = "Tomatoes"
resultados = buscar_conteudo(palavra_chave)

# Exibir os primeiros 500 caracteres de cada site
print("\nResultados da busca:\n")
for titulo, conteudo in resultados:
    print(f"{titulo}\n{conteudo[:500]}...\n")  # Mostra um trecho do conteúdo

# Fechar a conexão com o banco de dados
conn.close()

DB criado!

 Extraindo dados de: https://eos.com/pt/blog/como-plantar-tomates/
Como Plantar Tomate: Condições E Métodos De Cultivo - Usamos cookies essenciais para garantir que nosso site funcione corretamente. Para melhorar a experiência do usuário e a análise aprimorada do tráfego do site, também podemos usar cookies não essenciais, com o seu consentimento. Ao clicar em “Aceitar”, você concorda automaticamente com o uso de noss...

Dados inseridos com sucesso em https://eos.com/pt/blog/como-plantar-tomates/

 Extraindo dados de: https://www.jardineiro.net/como-plantar-tomate-os-segredos-do-plantio-ate-a-colheita.html
COMO PLANTAR TOMATE: OS SEGREDOS DO PLANTIO ATÉ A COLHEITA - Guia de Jardinagem
Atualizado em 21 de janeiro de 2024
Certamente todos sabemos que os tomates são um dos alimentos mais afetados por agrotóxicos. Segundo a ANVISA (Agência Nacional de Vigilância Sanitária) os tomates possuem resíduos acima do permitido. Mas existe uma saída, plantar tomate na sua c...

Dados i

Conversão dos dados SQL para JSON

In [35]:
import sqlite3
import json

# Conectar ao banco de dados
conn = sqlite3.connect("chatbot_jardinagem_db")
cursor = conn.cursor()

# Buscar conteúdos do BD
cursor.execute("SELECT titulo, conteudo FROM artigos")
dados = cursor.fetchall()

# Gerar dataset de perguntas e respostas
dataset = []

for titulo, conteudo in dados:
    pergunta = f"O que você pode me dizer sobre {titulo.lower()}?"
    resposta = conteudo[:500]  # Pegamos um trecho relevante do conteúdo

    dataset.append({"pergunta": pergunta, "resposta": resposta})

# Salvar como JSON
with open("dataset_jardinagem.json", "w", encoding="utf-8") as f:
    json.dump(dataset, f, indent=4, ensure_ascii=False)

print("✅ Dataset criado com sucesso!")


✅ Dataset criado com sucesso!


In [27]:
import json

# Carregar o arquivo JSON
with open("dataset_jardinagem.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

# Mostrar as primeiras entradas
for i, entrada in enumerate(dataset[:5]):  # Mostra as 5 primeiras entradas
    print(f"Pergunta: {entrada['pergunta']}")
    print(f"Resposta: {entrada['resposta']}\n")


Pergunta: O que você pode me dizer sobre how to grow tomatoes?
Resposta: Our detailed growing guide will help you with each step in successfully growing Tomatoes.
Cherry tomatoes are sweet and delicious, especially when harvested from your own plants
Tomatoes are one of the most popular crops to grow – they can be started from seed indoors or bought as young plants, then grown either outdoors or in a greenhouse, in pots or in the ground. They grow particularly well in a greenhouse, usually producing an earlier and larger crop. When grown outdoors, these tender plants

Pergunta: O que você pode me dizer sobre how to grow tomato plants: the complete guide?
Resposta: ORDER YOUR ALMANAC TODAY!
The Almanac Garden Planner - Use It Free for 7 Days!
Plan your 2025 garden with our award-winning Garden Planner.
TRY NOW
Are you growing garden-fresh tomatoes this year? There is simply nothing like them! Learn all about how to grow tomatoes, starting tomatoes, different varieties, and planting date

In [36]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
output_dir = "/content/drive/MyDrive/FlanT5_outputs"


Treinamento do FLAN-Y5

In [39]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset
import torch
import json

# Carregar o dataset JSON
with open("dataset_jardinagem.json", "r", encoding="utf-8") as f:
    dados = json.load(f)

# Converter para Dataset do Hugging Face
dataset = Dataset.from_list(dados)

# Dividir o dataset em treino e teste (80% treino e 20% teste)
dataset_treino = dataset.train_test_split(test_size=0.2)

# Carregar modelo e tokenizador
modelo_nome = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(modelo_nome)
model = T5ForConditionalGeneration.from_pretrained(modelo_nome)

# Tokenizar os dados
def preprocessar_amostra(exemplo):
    # Para garantir que cada item de "pergunta" e "resposta" seja corretamente manipulado como string
    entrada = ["Pergunta: " + str(p) + " Resposta:" for p in exemplo["pergunta"]]
    saida = [str(s) for s in exemplo["resposta"]]

    entrada_tokenizada = tokenizer(entrada, padding="max_length", truncation=True, max_length=512)
    saida_tokenizada = tokenizer(saida, padding="max_length", truncation=True, max_length=128)

    entrada_tokenizada["labels"] = saida_tokenizada["input_ids"]

    return entrada_tokenizada


# Aplicar a função com o parâmetro `batched=True`
dataset_treino = dataset_treino.map(preprocessar_amostra, batched=True)

# Configurar os parâmetros de treinamento
train_args = Seq2SeqTrainingArguments(
    output_dir="./modelo-ajustado",  # Pasta onde o modelo será salvo
    evaluation_strategy="epoch",  # Avaliar a cada época
    save_strategy="epoch",  # Salvar o modelo a cada época
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,  # Número de épocas
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=3,  # Salvar apenas os últimos 3 checkpoints
)

# Criar Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=train_args,
    train_dataset=dataset_treino["train"],  # Agora existe a chave 'train' após a divisão
    eval_dataset=dataset_treino["test"],  # Agora existe a chave 'test' após a divisão
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
)

# Iniciar o treinamento
trainer.train()

# Salvar o modelo treinado
model.save_pretrained("./modelo-ajustado")
tokenizer.save_pretrained("./modelo-ajustado")

print("✅ Fine-tuning concluído e modelo salvo!")




Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-39-68535473c88f>:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,No log,3.760436
2,No log,3.699226
3,No log,3.648565
4,No log,3.604293
5,No log,3.566322
6,No log,3.535548
7,No log,3.514266
8,No log,3.497639
9,No log,3.486574
10,No log,3.481529


✅ Fine-tuning concluído e modelo salvo!


In [42]:
# Carregar o modelo treinado
modelo_treinado = T5ForConditionalGeneration.from_pretrained("./modelo-ajustado")
tokenizer_treinado = T5Tokenizer.from_pretrained("./modelo-ajustado")

def responder_pergunta(pergunta):
    entrada = f"Pergunta: {pergunta} Resposta:"
    entrada_tokenizada = tokenizer_treinado(entrada, return_tensors="pt")

    # Gerar resposta
    saida_tokenizada = modelo_treinado.generate(**entrada_tokenizada, max_length=150)
    resposta = tokenizer_treinado.decode(saida_tokenizada[0], skip_special_tokens=True)

    return resposta

# Testar
pergunta = "Como plantar tomate?"
print(responder_pergunta(pergunta))




In [21]:
import os
print(os.listdir("./modelo-ajustado"))


['special_tokens_map.json', 'checkpoint-10', 'checkpoint-9', 'config.json', 'tokenizer_config.json', 'model.safetensors', 'checkpoint-8', 'generation_config.json', 'spiece.model', 'added_tokens.json']


In [23]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch


modelo_treinado_path = "./modelo-ajustado"

# Carregar o modelo e o tokenizador treinados
tokenizer = T5Tokenizer.from_pretrained(modelo_treinado_path)
model = T5ForConditionalGeneration.from_pretrained(modelo_treinado_path)

 # Criação da função para pré-processamento das perguntas, alimentar o FLAN-T5 e gerar uma resposta.
def gerar_resposta(pergunta, contexto):
    prompt = f"""
    You are an expert in gardening..
    Answer the question clearly and objectively based only on the context below:

    Context:
    {contexto}

    Question: {pergunta}
    Question:
    """

    # Tokenização correta com o prompt completo
    inputs = tokenizer(prompt, return_tensors="pt")

    # Gerar a resposta com temperatura controlada para evitar repetições
    resposta_ids = model.generate(
        **inputs,
        max_length=256,  # Aumentar o comprimento máximo da resposta
        max_new_tokens=512,
        num_beams=16,     # Usar mais "beams" para melhorar a busca
        temperature=0.7, # Controlar a criatividade (valores menores = mais determinístico)
        top_p=0.9,       # Usar amostragem com nucleus sampling
        early_stopping=True,
        no_repeat_ngram_size=2  # Evitar repetições de 3+ palavras
    )
    # Decodificar a resposta
    resposta = tokenizer.batch_decode(resposta_ids[0], skip_special_tokens=True)


    return resposta

def buscar_e_responder(pergunta):
    # Conectar ao banco de dados
    conn = sqlite3.connect("chatbot_jardinagem_db")
    cursor = conn.cursor()

    # Dividir a pergunta em palavras-chave
    palavras_chave = pergunta.lower().split()

    # Criar a consulta SQL mais eficiente
    query = """
    SELECT titulo, conteudo FROM artigos
    WHERE conteudo LIKE ?
    ORDER BY LENGTH(conteudo) ASC
    LIMIT 3
    """
    valores = [f"%{pergunta}%"]  # Buscar a pergunta completa primeiro

    cursor.execute(query, valores)
    resultados = cursor.fetchall()

    conn.close()

    if not resultados:
        return "Não encontrei informações suficientes sobre isso."

    # Selecionar os 500 primeiros caracteres do melhor resultado
    contexto = "\n".join([f"{titulo}: {conteudo[:500]}" for titulo, conteudo in resultados])

    # Gerar resposta com FLAN-T5
    resposta = gerar_resposta(pergunta, contexto)

    return resposta



pergunta = "how to plant tomatoes?"
resposta = buscar_e_responder(pergunta)
print("🤖 Chatbot:", resposta)

🤖 Chatbot: Não encontrei informações suficientes sobre isso.


In [41]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

modelo_treinado_path = "./modelo-ajustado"

# Carregar o modelo e o tokenizador treinados
tokenizer = T5Tokenizer.from_pretrained(modelo_treinado_path)
model = T5ForConditionalGeneration.from_pretrained(modelo_treinado_path)

inputs = tokenizer("how to plant tomatoes?:", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))



['Place a sprig of tomato in a pot and cover with a layer of']
